In [ ]:
import csv

class VeganStore:
    def __init__(self, inventory):        
        """
        
        Initialize a VeganStore object.

        Parameters:
        - inventory (str): The filename of the CSV file containing inventory information.
        
        """
        self.inventory = inventory
        self.create_inv()  
        self.products = self.load_products_from_csv()
        self.sales = []

    def create_inv(self):          
        """
        
        Create the inventory file if it does not exist or check if it exists.
        
        """
        try:
            with open(self.inventory, 'r', newline=''):
                pass  
        except FileNotFoundError:
            with open(self.inventory, 'w', newline='') as file:
                csv_writer = csv.DictWriter(file, fieldnames=['Nome', 'Quantità', 'PrezzoAcquisto', 'PrezzoVendita'], delimiter=';')
                csv_writer.writeheader()

    def load_products_from_csv(self):        
        """
        Load product information from a CSV file into a list of dictionaries.
        
        """
        
        try:
            with open(self.inventory, 'r', newline='') as file:
                csv_reader = csv.DictReader(file, delimiter=';')
                products = []

                for row in csv_reader:
                    product = {
                        'Nome': row['Nome'],
                        'Quantità': int(row['Quantità']),
                        'PrezzoAcquisto': float(row['PrezzoAcquisto']),
                        'PrezzoVendita': float(row['PrezzoVendita'])}
                    products.append(product)
                return products
        except FileNotFoundError:
            print(f"Il file {self.inventory} non esiste.")
            return []

    def save_products_to_csv(self):        
        """
        
        Save product information to the CSV file.
        
        """
        with open(self.inventory, 'w', newline='') as file:
            csv_writer = csv.DictWriter(file, fieldnames=['Nome', 'Quantità', 'PrezzoAcquisto', 'PrezzoVendita'], delimiter=';')
            csv_writer.writeheader()
            csv_writer.writerows(self.products)

    def is_valid_input(self, value, type_value):        
        """
        
        Validate user input based on the specified data type.

        Parameters:
        - value: The user input to be validated.
        - type_value (str): The expected data type ("int" or "float").
        
        """
        
        str_type_value = ""
        while True:
            try:
                if type_value == "int":
                    str_type_value = "intero"
                    value = int(value)
                elif type_value == "float":
                    str_type_value = "intero o decimale"
                    value = float(value)

                if value < 0:
                    raise ValueError(f"I valori negativi non sono consentiti. Inserisci un valore {str_type_value} positivo.")

                return value
            except ValueError:
                print(f"Input non valido. Inserisci un valore {str_type_value} valido.")
                value = input(f"Inserisci un valore {str_type_value} valido: ")

    def add_product(self, name):        
        """
        
        Add a new product to the inventory.

        Parameters:
        - name (str): The name of the product.
        
        """    
        
        quantity = input("Quantità: ")
        quantity = self.is_valid_input(quantity, "int")

        for product in self.products:
            if product['Nome'] == name:
                product['Quantità'] += quantity
                print(f"AGGIUNTO: {quantity} X {name}\n")
                self.save_products_to_csv()
                return

        purchase_price = input("Prezzo di acquisto: ")
        purchase_price = self.is_valid_input(purchase_price, "float")
        selling_price = input("Prezzo di vendita: ")
        selling_price = self.is_valid_input(selling_price, "float")

        new_product = {
            'Nome': name,
            'Quantità': quantity,
            'PrezzoAcquisto': purchase_price,
            'PrezzoVendita': selling_price}
        self.products.append(new_product)
        print(f"AGGIUNTO: {quantity} X {name}\n")
        self.save_products_to_csv()
        return new_product

    def list_products(self):        
        """
        
        List all products in the inventory.
        
        """
        
        if not self.products:
            print("Il magazzino è vuoto.")
        else:
            for product in self.products:
                print(f"{product['Nome']} - Quantità: {product['Quantità']}, Prezzo di vendita: €{product['PrezzoVendita']:.2f}")

                
    def record_sale(self, product_name, sold_quantity):        
        """
        
        Record a sale of a product.

        Parameters:
        - product_name (str): The name of the product being sold.
        - sold_quantity: The quantity of the product being sold.
        
        """
        
        sale_summary = []

        while True:
            for product in self.products:
                if product['Nome'] == product_name:
                    sold_quantity = self.is_valid_input(sold_quantity, "int")
                    if product['Quantità'] >= sold_quantity:
                        product['Quantità'] -= sold_quantity
                        sale = sold_quantity * product['PrezzoVendita']
                        sale_summary.append((product_name, sold_quantity, sale))
                        self.sales.append({'Productname': product_name, 'Soldquantity': sold_quantity, 'Totalprice': sale})
                        print(f"VENDITA REGISTRATA\n- {sold_quantity} X {product_name}: €{sale:.2f}")
                        self.save_products_to_csv()
                        add_another_sale = input("Aggiungere un altro prodotto da vendere? (si/no): ")

                        if add_another_sale.lower() == 'si':
                            product_name = input("Nome del prodotto: ")
                            sold_quantity = input("Quantità venduta: ")
                            continue

                        print("\nDettagli della vendita:")
                        for product_name, quantity, sale_price in sale_summary:
                            print(f"- {quantity} X {product_name}: €{sale_price:.2f}")

                        total_price = sum(sale_price for _, _, sale_price in sale_summary)
                        print(f"\nTotale: €{total_price:.2f}\n")
                        return

                    else:
                        print(f"Quantità insufficiente di {product_name} in magazzino.")
                        return

            print(f"Prodotto {product_name} non trovato.")
            return

    def show_profits(self):        
        """
        
        Display total gross and net profits.
        
        """
        
        total_gross_profit = 0
        total_net_profit = 0

        for sale in self.sales:
            total_gross_profit += sale['Totalprice']
            for product in self.products:
                if product['Nome'] == sale['Productname']:
                    total_net_profit += sale['Totalprice'] - (product['PrezzoAcquisto'] * sale['Soldquantity'])

        print(f"Profitti lordi: €{total_gross_profit:.2f}, Profitti netti: €{total_net_profit:.2f}")

    def show_help_menu(self):        
        """
        
        Display the available commands in the help menu.
        
        """
        
        print("""Comandi disponibili:
- aggiungi: aggiungi un prodotto all'inventario
- lista: elenca i prodotti in magazzino
- vendita: registra una vendita
- profitti: mostra i profitti totali
- aiuto: mostra i comandi disponibili
- esci: esci dal programma""")

    def cmd_program(self):        
        """
        
        Run the main program loop for user interaction.
        
        """
        
        print("Benvenuto!")

        cmd = None

        while cmd != "esci":
            cmd = input("Inserisci un comando (aiuto per la lista completa): ")

            if cmd == 'aggiungi':
                while True:
                    name = input("Nome prodotto: ")
                    self.add_product(name)

                    add_another_product = input("Aggiungere un altro prodotto? (si/no): ")
                    if add_another_product.lower() != 'si':
                        break

            elif cmd == 'lista':
                self.list_products()

            elif cmd == 'vendita':
                product_name = input("Nome prodotto: ")
                sold_quantity = input("Quantità venduta: ")
                self.record_sale(product_name, sold_quantity)

            elif cmd == 'profitti':
                self.show_profits()

            elif cmd == 'aiuto':
                self.show_help_menu()

            elif cmd == 'esci':
                print("Bye Bye")
                break

            else:
                print("Comando non valido. Inserisci 'aiuto' per la lista completa dei comandi.")


vegan_store = VeganStore("magazzino.csv")
vegan_store.cmd_program()

Benvenuto!
Inserisci un comando (aiuto per la lista completa): lista
Il magazzino è vuoto.
